# Packages

In [1]:
!pip install atproto

In [2]:
from atproto import Client
import json
from datetime import datetime, timezone
import os

# Functions

In [3]:
def save_json(data,output_folder,file_name):
  os.makedirs(output_folder, exist_ok=True)
  with open(output_folder+'/'+file_name, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


In [4]:
def save_posts_json(result,query,cursor):
  output_folder=query
  os.makedirs(output_folder, exist_ok=True)
  data = result.model_dump()

  if cursor is None:
    file_name = f'posts_0.json'
  else:
    file_name = f'posts_{cursor}.json'

  save_json(data,output_folder+'/posts',file_name)

In [5]:
def save_threads_json(result,query,root_id):
    output_folder=query
    os.makedirs(output_folder, exist_ok=True)
    data = result.model_dump()

    file_name = f'thread_{root_id}.json'

    save_json(data,output_folder+'/threads',file_name)


In [6]:
def find_root_uri(post):
    try:
      return post["record"]["reply"]["root"]["uri"]
    except (KeyError, TypeError):
      return post["uri"]

# Client

In [7]:
user_name = ''
user_password = ''

client = Client()
client.login(user_name, user_password)

ProfileViewDetailed(did='did:plc:gwar7tkxodqgvm3xema7dekb', handle='aleixnicolas.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:gwar7tkxodqgvm3xema7dekb/bafkreier6ooh65m5cfkmueogubhpko64jwwe2gjc7yay7vuqjfjoxiamiq@jpeg', banner=None, created_at='2025-05-13T10:31:03.440Z', description=None, display_name='', followers_count=6, follows_count=1, indexed_at='2025-05-13T10:42:17.040Z', joined_via_starter_pack=None, labels=[Label(cts='2025-05-13T10:31:03.566Z', src='did:plc:gwar7tkxodqgvm3xema7dekb', uri='at://did:plc:gwar7tkxodqgvm3xema7dekb/app.bsky.actor.profile/self', val='!no-unauthenticated', cid='bafyreicijhetkuql7vfuinv6bgibrp4ljdf54ukyjt3jiphzskgnctlynm', exp=None, neg=None, sig=None, ver=None, py_type='com.atproto.label.defs#label')], pinned_post=None, posts_count=0, verification=None, viewer=ViewerState(blocked_by=False, 

# Post query

In [8]:
# Search and save posts with corresponding query, can be modified to sort by date and
# add additional filters.

query='parking bonaire'
max_cursor=5
cursor = None
for i in range(max_cursor):
  result = client.app.bsky.feed.search_posts({'q': query
                                            ,'sort': 'top'
                                            #,'since': start_date
                                            #,'unitl':end_date
                                            #,'mentions':
                                            #,'author':
                                            #,'lang':
                                            #,'domain':
                                            #,'url':
                                            #,'tag':
                                            ,'limit': 100
                                            ,'cursor': cursor
                                            })
  save_posts_json(result,query,cursor)
  cursor = result.cursor
  if cursor is None:
    break


# Thread query

In [9]:
query='parking bonaire'
posts_dir = os.path.join(query, 'posts')
unique_root_uris = set()

for filename in os.listdir(posts_dir):
  if filename.endswith('.json'):
    filepath = os.path.join(posts_dir, filename)
    with open(filepath, "r", encoding="utf-8") as f:
      posts = json.load(f)
      for post in posts['posts']:
        root_uri = find_root_uri(post)
        unique_root_uris.add(root_uri)

In [10]:
for i, root_uri in enumerate(unique_root_uris):
  try:
    result = client.app.bsky.feed.get_post_thread({'uri': root_uri,'depth': 1000})
    save_threads_json(result,query,i)
  except:
    print('Thread not found: '+root_uri)

Thread not found: at://did:plc:rmwt56fehcqb4qbfgpo6ayie/app.bsky.feed.post/3la3a5ychq32x
Thread not found: at://did:plc:mhstredor32uwsfajpoemgzf/app.bsky.feed.post/3lb3g7loork2w
Thread not found: at://did:plc:sykhf5hlsdfuvaxfedmy32io/app.bsky.feed.post/3lb6l7pwty22r
Thread not found: at://did:plc:swawo27jb75lygtiijxbszzp/app.bsky.feed.post/3l7xkqvffc72r
